In [91]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

In [36]:
print(os.getcwd())
print(os.listdir())

/Users/barbem4/Desktop/Vandy/Bioinformatics
['.DS_Store', 'ancestry_map_1k_var.pkl', 'Ancestry prediction - BMIF6310.ipynb', '.ipynb_checkpoints']


In [37]:
data_df = pd.read_pickle('ancestry_map_1k_var.pkl')
data_df

,POS,REF,ALT,INFO,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,NaN,NaN,NaN,NaN,EUR,EUR,EUR,EUR,EUR,EUR,...,SAS,SAS,SAS,SAS,SAS,SAS,SAS,SAS,SAS,SAS
1,10416,CCCTAA,C,AC=240;AN=5096;DP=365460;AF=0.05;EAS_AF=0.06;E...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,16103,T,G,AC=118;AN=5096;DP=29994;AF=0.02;EAS_AF=0;EUR_A...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,17496,AC,A,AC=25;AN=5096;DP=189765;AF=0;EAS_AF=0;EUR_AF=0...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,51479,T,A,AC=531;AN=5096;DP=17461;AF=0.1;EAS_AF=0;EUR_AF...,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,839006,CAG,C,AC=8;AN=5096;DP=16336;AF=0;EAS_AF=0;EUR_AF=0;A...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,839011,G,T,AC=31;AN=5096;DP=17353;AF=0.01;EAS_AF=0;EUR_AF...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,839166,TGAGA,T,AC=110;AN=5096;DP=18786;AF=0.02;EAS_AF=0;EUR_A...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
999,839201,G,GA,AC=38;AN=5096;DP=19102;AF=0.01;EAS_AF=0;EUR_AF...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
X = data_df.iloc[1:,4:].values.T
X.shape

(2548, 1000)

In [39]:
y = data_df.iloc[0,4:].values
y.shape

(2548,)

## Bernouilli Naive Bayes

Bernoulli Naive Bayes — adaptation for boolean attributes
If you have binary-valued attributes (Bernoulli, boolean), then you can use a Bernoulli NB model.

Naive Bayes' underlying assumption is that the predictors (attributes / independent variables) are independent of each other. This is a big assumption because it is easy to show that there is often at least some correlation between variables in real life. It is precisely this assumption of independence that makes Bayes classification “naive.

In [40]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [41]:
#Split dataset 70/20/10: train/val/test
X_train, X_test, y_train, y_test = train_test_split(X,y.T, train_size=0.70, shuffle = True)

X_train.shape

(1783, 1000)

In [42]:
#Alpha hyperparameter (laplacian smoothing) does not really affect this task as long as it is close to zero -> we dont artificially impact probabilities coming from data
#Avoid zero as this could give null posterior probabilities as a result of some zero likelihood probabilities coming from lack of some fearture-class example in trainign datset
Bnb = BernoulliNB(alpha = 0.0001, binarize = 0.5)

In [43]:
Bnb.fit(X_train, y_train)

BernoulliNB(alpha=0.0001, binarize=0.5)

In [44]:
# Predict class labels on a test data
pred_labels = Bnb.predict(X_test)

# Print model performance sumamry
print(f"n\n\Classification report on test set for classifier {Bnb}:\n"
f"{metrics.classification_report(y_test,pred_labels)}\n")

n
\Classification report on test set for classifier BernoulliNB(alpha=0.0001, binarize=0.5):
              precision    recall  f1-score   support

         AFR       0.94      0.92      0.93       224
         AMR       0.62      0.62      0.62        97
         EAS       0.86      0.99      0.92       134
         EUR       0.66      0.77      0.71       149
         SAS       0.86      0.66      0.75       161

    accuracy                           0.81       765
   macro avg       0.79      0.79      0.79       765
weighted avg       0.82      0.81      0.81       765




In [45]:
counting = list(y_test[pred_labels!=y_test]).count
totals_count = list(y_test).count
labels = Bnb.classes_

In [46]:
#Error/False negative rate for each class: 1-recall
FNR =[np.round(100*counting(x)/totals_count(x),decimals=2) for x in Bnb.classes_]
print(Bnb.classes_,"\n",FNR,"%FNR")

['AFR' 'AMR' 'EAS' 'EUR' 'SAS'] 
 [8.04, 38.14, 1.49, 22.82, 33.54] %FNR


In [47]:
results = data_df.iloc[1:,0:4]
results

,POS,REF,ALT,INFO
1,10416,CCCTAA,C,AC=240;AN=5096;DP=365460;AF=0.05;EAS_AF=0.06;E...
2,16103,T,G,AC=118;AN=5096;DP=29994;AF=0.02;EAS_AF=0;EUR_A...
3,17496,AC,A,AC=25;AN=5096;DP=189765;AF=0;EAS_AF=0;EUR_AF=0...
4,51479,T,A,AC=531;AN=5096;DP=17461;AF=0.1;EAS_AF=0;EUR_AF...
5,51898,C,A,AC=426;AN=5096;DP=15331;AF=0.08;EAS_AF=0.05;EU...
...,...,...,...,...
996,839006,CAG,C,AC=8;AN=5096;DP=16336;AF=0;EAS_AF=0;EUR_AF=0;A...
997,839011,G,T,AC=31;AN=5096;DP=17353;AF=0.01;EAS_AF=0;EUR_AF...
998,839166,TGAGA,T,AC=110;AN=5096;DP=18786;AF=0.02;EAS_AF=0;EUR_A...
999,839201,G,GA,AC=38;AN=5096;DP=19102;AF=0.01;EAS_AF=0;EUR_AF...


In [120]:
relevance = 10**Bnb.feature_log_prob_
relevance.shape

(5, 1000)

In [121]:
for i in range(5):
    results[labels[i]] = relevance[i]

In [122]:
results

,POS,REF,ALT,INFO,AFR,AMR,EAS,EUR,SAS
1,10416,CCCTAA,C,AC=240;AN=5096;DP=365460;AF=0.05;EAS_AF=0.06;EUR_AF=0.07;AFR_AF=0.01;AMR_AF=0.06;SAS_AF=0.05;VT=INDEL;NS=2548,2.411881e-04,8.100964e-03,8.480143e-03,1.546310e-02,3.972663e-03
2,16103,T,G,AC=118;AN=5096;DP=29994;AF=0.02;EAS_AF=0;EUR_AF=0.04;AFR_AF=0.03;AMR_AF=0.03;SAS_AF=0.01;VT=SNP;NS=2548,3.026653e-03,1.766612e-03,1.141038e-06,3.017323e-03,3.165746e-04
3,17496,AC,A,AC=25;AN=5096;DP=189765;AF=0;EAS_AF=0;EUR_AF=0;AFR_AF=0.02;AMR_AF=0;SAS_AF=0;VT=INDEL;NS=2548,6.948406e-04,1.837409e-15,7.028453e-16,7.380413e-16,9.717189e-16
4,51479,T,A,AC=531;AN=5096;DP=17461;AF=0.1;EAS_AF=0;EUR_AF=0.19;AFR_AF=0.02;AMR_AF=0.11;SAS_AF=0.23;VT=SNP;NS=2548,2.900000e-04,2.549024e-02,2.776639e-05,9.468073e-02,1.183257e-01
5,51898,C,A,AC=426;AN=5096;DP=15331;AF=0.08;EAS_AF=0.05;EUR_AF=0.14;AFR_AF=0.06;AMR_AF=0.06;SAS_AF=0.11;VT=SNP;NS=2548,5.322118e-03,8.715359e-03,4.372419e-03,3.410119e-02,2.192892e-02
...,...,...,...,...,...,...,...,...,...
996,839006,CAG,C,AC=8;AN=5096;DP=16336;AF=0;EAS_AF=0;EUR_AF=0;AFR_AF=0;AMR_AF=0;SAS_AF=0;VT=INDEL;NS=2548,4.865199e-16,1.837409e-15,7.028453e-16,5.910418e-06,1.979368e-05
997,839011,G,T,AC=31;AN=5096;DP=17353;AF=0.01;EAS_AF=0;EUR_AF=0;AFR_AF=0.02;AMR_AF=0;SAS_AF=0;VT=SNP;NS=2548,7.819486e-04,1.837409e-15,7.028453e-16,7.380413e-16,9.717189e-16
998,839166,TGAGA,T,AC=110;AN=5096;DP=18786;AF=0.02;EAS_AF=0;EUR_AF=0;AFR_AF=0.08;AMR_AF=0.01;SAS_AF=0;VT=INDEL;NS=2548,1.399386e-02,7.258813e-05,7.028453e-16,5.910418e-06,9.717189e-16
999,839201,G,GA,AC=38;AN=5096;DP=19102;AF=0.01;EAS_AF=0;EUR_AF=0;AFR_AF=0.03;AMR_AF=0;SAS_AF=0;VT=INDEL;NS=2548,1.989032e-03,1.837409e-15,7.028453e-16,7.380413e-16,9.717189e-16


In [144]:
results['SAS'].iloc[np.argsort(results[ancestry].values,order='')[-20:]]


ValueError: Cannot specify order when the array has no fields.

In [138]:
#Provide the top 20 most important mutations to predict an ancestry class
ancestry = 'SAS'

final = results.iloc[np.argsort(results[ancestry].values)[-20:]]

final['INFO'] = final['INFO'].str.replace(r'^(.*?)EAS_AF',r'EAS_AF',regex=True)

final.drop(list(labels[np.where(labels!=ancestry)]),axis=1)

<ipython-input-138-4d85a9770175>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['INFO'] = final['INFO'].str.replace(r'^(.*?)EAS_AF',r'EAS_AF',regex=True)


,POS,REF,ALT,INFO,SAS
685,820560,C,T,EAS_AF=0.89;EUR_AF=0.84;AFR_AF=0.44;AMR_AF=0.81;SAS_AF=0.81;VT=SNP;NS=2548,0.925129
689,821000,T,A,EAS_AF=0.89;EUR_AF=0.87;AFR_AF=0.45;AMR_AF=0.82;SAS_AF=0.82;VT=SNP;NS=2548,0.925129
730,822764,A,G,EAS_AF=0.89;EUR_AF=0.87;AFR_AF=0.45;AMR_AF=0.82;SAS_AF=0.83;VT=SNP;NS=2548,0.925129
746,823246,C,T,EAS_AF=0.89;EUR_AF=0.87;AFR_AF=0.45;AMR_AF=0.82;SAS_AF=0.83;VT=SNP;NS=2548,0.925129
695,821224,A,G,EAS_AF=0.89;EUR_AF=0.87;AFR_AF=0.46;AMR_AF=0.82;SAS_AF=0.83;VT=SNP;NS=2548,0.925129
722,822354,C,T,EAS_AF=0.89;EUR_AF=0.87;AFR_AF=0.45;AMR_AF=0.82;SAS_AF=0.83;VT=SNP;NS=2548,0.925129
945,837375,A,C,EAS_AF=0.9;EUR_AF=0.87;AFR_AF=0.45;AMR_AF=0.82;SAS_AF=0.83;VT=SNP;NS=2548,0.925129
569,817186,G,A,EAS_AF=0.89;EUR_AF=0.84;AFR_AF=0.39;AMR_AF=0.8;SAS_AF=0.81;VT=SNP;NS=2548,0.925129
762,824457,T,A,EAS_AF=0.89;EUR_AF=0.87;AFR_AF=0.46;AMR_AF=0.82;SAS_AF=0.84;VT=SNP;NS=2548,0.931799
684,820510,A,T,EAS_AF=0.89;EUR_AF=0.87;AFR_AF=0.46;AMR_AF=0.82;SAS_AF=0.84;VT=SNP;NS=2548,0.931799


In [130]:
#Understand coefficients in the model, make histogram and look into 